<a href="https://colab.research.google.com/github/giorgitorro/FER_gtoro22/blob/main/FER2013_ML_gtoro22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision torchaudio
!pip install wandb
!pip install kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"giorgitoronjadze","key":"82572ceeacdd135dce03ab6b1a5379a1"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!pip install opendatasets

In [5]:
import opendatasets as od

In [6]:
od.download("https://www.kaggle.com/competitions/challenges-in-representation-learning-facial-expression-recognition-challenge/overview")

100%|██████████| 285M/285M [00:00<00:00, 312MB/s]



Extracting archive ./challenges-in-representation-learning-facial-expression-recognition-challenge/challenges-in-representation-learning-facial-expression-recognition-challenge.zip to ./challenges-in-representation-learning-facial-expression-recognition-challenge


In [7]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: gtoro22 (gtoro22-free-university-of-tbilisi-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [8]:
!git init
!git config --global user.email "gtoro22@freeuni.edu.ge"
!git config --global user.name "giorgitorro"
!git remote add origin https://github.com/giorgitorro/FER_gtoro22.git

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/


In [9]:
import pandas as pd

df = pd.read_csv("/content/challenges-in-representation-learning-facial-expression-recognition-challenge/train.csv")
print(df.head())
print(df.columns)

   emotion                                             pixels
0        0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1        0  151 150 147 155 148 133 111 140 170 174 182 15...
2        2  231 212 156 164 174 138 161 173 182 200 106 38...
3        4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4        6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...
Index(['emotion', 'pixels'], dtype='object')


In [10]:
import numpy as np
from sklearn.model_selection import train_test_split

def preprocess_data(df):
    images = np.array([np.fromstring(row, dtype=int, sep=' ').reshape(48, 48) for row in df['pixels']])
    images = images.astype('float32') / 255.0
    images = np.expand_dims(images, axis=1)

    labels = df['emotion'].values

    return images, labels

X, y = preprocess_data(df)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


In [11]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# Convert numpy arrays to torch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

# Create Datasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)


In [12]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 12 * 12, 128)
        self.fc2 = nn.Linear(128, 7)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # (48→24)
        x = self.pool(F.relu(self.conv2(x)))  # (24→12)
        x = x.view(-1, 64 * 12 * 12)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [13]:
import wandb

# Initialize wandb
wandb.init(project="fer2013", name="simple_cnn_baseline")
wandb.config.update({
    "epochs": 10,
    "batch_size": 64,
    "learning_rate": 0.001,
    "architecture": "SimpleCNN"
})

# Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(wandb.config.epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total
    avg_loss = total_loss / len(train_loader)

    # Evaluate on validation set
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = outputs.argmax(dim=1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)
    val_acc = val_correct / val_total

    print(f"Epoch {epoch+1}: Train Acc = {train_acc:.4f}, Val Acc = {val_acc:.4f}, Loss = {avg_loss:.4f}")
    wandb.log({
        "train_accuracy": train_acc,
        "val_accuracy": val_acc,
        "loss": avg_loss,
        "epoch": epoch + 1
    })


Epoch 1: Train Acc = 0.3392, Val Acc = 0.3999, Loss = 1.6646
Epoch 2: Train Acc = 0.4284, Val Acc = 0.4457, Loss = 1.4873
Epoch 3: Train Acc = 0.4769, Val Acc = 0.4765, Loss = 1.3668
Epoch 4: Train Acc = 0.5124, Val Acc = 0.4974, Loss = 1.2773
Epoch 5: Train Acc = 0.5503, Val Acc = 0.5089, Loss = 1.1893
Epoch 6: Train Acc = 0.5868, Val Acc = 0.5138, Loss = 1.1056
Epoch 7: Train Acc = 0.6233, Val Acc = 0.5139, Loss = 1.0180
Epoch 8: Train Acc = 0.6613, Val Acc = 0.5230, Loss = 0.9231
Epoch 9: Train Acc = 0.6999, Val Acc = 0.5204, Loss = 0.8289
Epoch 10: Train Acc = 0.7414, Val Acc = 0.5152, Loss = 0.7272


In [14]:
!git add .
!git commit -m "Initial model training with SimpleCNN baseline"
!git push origin master

[master (root-commit) 5c610a1] Initial model training with SimpleCNN baseline
 37 files changed, 127157 insertions(+)
 create mode 100644 .config/.last_opt_in_prompt.yaml
 create mode 100644 .config/.last_survey_prompt.yaml
 create mode 100644 .config/.last_update_check.json
 create mode 100644 .config/active_config
 create mode 100644 .config/config_sentinel
 create mode 100644 .config/configurations/config_default
 create mode 100644 .config/default_configs.db
 create mode 100644 .config/gce
 create mode 100644 .config/hidden_gcloud_config_universe_descriptor_data_cache_configs.db
 create mode 100644 .config/logs/2025.06.04/21.28.54.286615.log
 create mode 100644 .config/logs/2025.06.04/21.29.15.722593.log
 create mode 100644 .config/logs/2025.06.04/21.29.24.677039.log
 create mode 100644 .config/logs/2025.06.04/21.29.26.203008.log
 create mode 100644 .config/logs/2025.06.04/21.29.34.884405.log
 create mode 100644 .config/logs/2025.06.04/21.29.35.667854.log
 create mode 100644 challe